In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [18]:
from adversarial.defences.adv_classifier import *
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from tqdm import tqdm

In [19]:
import time

In [4]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import numpy as np

In [6]:
# dataloader
from adversarial.utils import dataloader
from adversarial.attacks import random_attack

In [7]:
BATCH_SIZE = 32
EPSILONS = np.arange(0.01, 0.5, step=0.02)
ETA = 1e-4
EPOCHS = 10

In [8]:
train_data_loader = dataloader.DataLoader('./data', BATCH_SIZE, training=True)
test_data_loader = dataloader.DataLoader('./data', BATCH_SIZE, training= False)

In [9]:
from adversarial.defences import EnsembleModel
from adversarial.models import *

random_attack(앙상블 모델 넣을건데, x_image, adversarial_label, epsilon은 배열로)  
epsilon = np.arange(0.01,0.5,0.02)  
random_attack은 adversarial image를 리턴해준다. 

In [10]:
target_model= AdvClassifierMk2()

In [11]:
# train

criterion = losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = ETA)

train_losses = tf.keras.metrics.CategoricalCrossentropy()
train_metrics = tf.keras.metrics.CategoricalAccuracy()

test_losses = tf.keras.metrics.CategoricalCrossentropy()
test_metrics = tf.keras.metrics.CategoricalAccuracy()

In [12]:
ensemble_model = EnsembleModel([
    VGG19, VGG16, MobileNet, MobileNetV2, ResNet50, ResNet50V2
])

In [13]:
labels = [[0,1] ,[1,0]]
    
y_batch = []
    
    
for label in labels : 
    for time in range(BATCH_SIZE) :
        y_batch.append(label)

y_tensor = tf.convert_to_tensor(y_batch)

In [14]:
def train_step(images, adv_labels):

    adv, _, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    
    x_batch = tf.concat([images, adv], axis=0)
    y_batch = y_tensor
    with tf.GradientTape() as tape:
        outputs = target_model(x_batch, training = True)
      #  print('y_batch : ' ,y_batch)
       # print('outputs : ' ,outputs)
        loss = criterion(y_batch, outputs)
    print('batch loss : {0}'.format(loss.numpy()), end = '\r')
    train_losses.update_state(y_batch, outputs)
    train_metrics.update_state(y_batch, outputs)
        
    grads = tape.gradient(loss, target_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, target_model.trainable_variables))

In [15]:
def test_step(images, labels, adv_labels) :
    adv,_, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    x_batch = tf.concat([images, adv], axis=0)
    y_batch = y_tensor
    #y_batch = tf.concat([tf.zeros(16, dtype = tf.int32),
     #                    tf.ones(16, dtype = tf.int32)], 
      #                   axis = 0)
    
    predictions = target_model(x_batch, training= False)
    loss = tf.keras.losses.binary_crossentropy(y_batch, predictions)
    test_losses.update_state(y_batch, predictions)
    test_metrics.update_state(y_batch, predictions)

In [23]:
def train():

    train_loss_list = []
    test_loss_list = []

    for e in range(EPOCHS):
        start = time.time()
        for x_batch, y_batch, adv_batch in iter(train_data_loader):
            train_step(x_batch, adv_batch)

        for x_batch, y_batch, adv_batch in iter(test_data_loader):
            test_step(x_batch, y_batch, adv_batch)


        train_loss = train_losses.result()
        train_acc = train_metrics.result()

        test_loss = test_losses.result()
        test_acc = test_metrics.result()

        train_loss_list.append(train_loss)
        test_loss_list.append(test_loss)

        train_losses.reset_states()
        train_metrics.reset_states()
        test_losses.reset_states()
        test_metrics.reset_states()
        end = time.time()
        print(f"Epochs {e+1}/{EPOCHS}, train loss: {train_loss:.8f}, train acc: {train_acc:.4f}, test loss: {test_loss:.8f}, test acc: {test_acc:.4f}, time_cost: {end-start}s")

In [24]:
train()

Epochs 1/10, train loss: 0.63947999, train acc: 0.5890, test loss: 0.49207914, test acc: 0.7383, time_cost: 3100.7824914455414s
Epochs 2/10, train loss: 0.61510414, train acc: 0.6086, test loss: 0.62301821, test acc: 0.5957, time_cost: 3245.1482121944427s
Epochs 3/10, train loss: 0.59973454, train acc: 0.6357, test loss: 0.53303570, test acc: 0.7002, time_cost: 3219.652215719223s
Epochs 4/10, train loss: 0.58941013, train acc: 0.6429, test loss: 0.54221088, test acc: 0.6992, time_cost: 2477.8454439640045s
Epochs 5/10, train loss: 0.59811044, train acc: 0.6309, test loss: 0.50061947, test acc: 0.7461, time_cost: 3449.925002336502s
Epochs 6/10, train loss: 0.58034539, train acc: 0.6530, test loss: 0.59827918, test acc: 0.6240, time_cost: 2753.243672132492s
Epochs 7/10, train loss: 0.57868254, train acc: 0.6544, test loss: 0.57340789, test acc: 0.6572, time_cost: 2339.77823138237s
Epochs 8/10, train loss: 0.57428002, train acc: 0.6583, test loss: 0.62551349, test acc: 0.5938, time_cost: 2

In [ ]:
train()

In [ ]:
train()

In [ ]:
save_custom_weights(target_model, './model/adv_classifier_mk2.pkl')